# Single-Mode Cavity with TLS

Demonstrate `maxwelllink.SingleModeSimulation` coupled to TLS models in both
embedded and UNIX-socket configurations.

## 1. Embedded TLS (no socket)

In [ ]:
import numpy as np
import maxwelllink as mxl

single_tls = mxl.Molecule(
    driver="tls",
    driver_kwargs=dict(omega=0.242, mu12=187.0, orientation=2, pe_initial=1e-3),
)

sm_sim = mxl.SingleModeSimulation(
    dt_au=0.05,
    frequency_au=0.242,
    damping_au=1e-3,
    molecules=[single_tls],
    drive=lambda t: 1e-4 * np.sin(0.242 * t),
    coupling_strength=1.0,
)

sm_sim.run(steps=400)

import matplotlib.pyplot as plt

plt.figure(figsize=(6, 4))
plt.plot(sm_sim.time_history, sm_sim.field_history, label="Field E")
plt.xlabel("time (a.u.)")
plt.ylabel("field amplitude (a.u.)")
plt.legend()
plt.tight_layout()
plt.show()

print("Final TLS population:", single_tls.additional_data_history[-1]["Pe"])


## 2. Coupling via UNIX Socket

Run the following helper to initialize the single-mode cavity while exposing a
UNIX domain socket. Launch `mxl_driver.py` in another shell with `--unix` to
connect the TLS driver.

In [ ]:
from maxwelllink import sockets as mxs

def run_single_mode_socket(steps=400):
    hub = mxl.SocketHub(unixsocket="single_mode_tls", timeout=10.0, latency=1e-5)

    molecule = mxl.Molecule(
        hub=hub,
        sigma=0.1,
        size=None,
        center=None,
        dimensions=None,
    )

    sim = mxl.SingleModeSimulation(
        dt_au=0.05,
        frequency_au=0.242,
        damping_au=1e-3,
        molecules=[molecule],
        drive=0.0,
        coupling_strength=1.0,
        hub=hub,
    )

    print("Launch the driver:")
    print("  mxl_driver.py --model tls --unix --address single_mode_tls --param "omega=0.242, mu12=187, orientation=2, pe_initial=1e-3"")

    sim.run(steps=steps)
    return sim, molecule


After running the driver, execute this cell to capture the cavity dynamics.

In [ ]:
# sim_socket, molecule_socket = run_single_mode_socket()
# Inspect the results as needed, for example:
# import matplotlib.pyplot as plt
# plt.plot(sim_socket.time_history, sim_socket.field_history)
# plt.xlabel("time (a.u.)")
# plt.ylabel("field amplitude (a.u.)")
# plt.show()
